# Create a forcing time series of aerosols and ozone

We want a forcing time series that we can implement directly into fair for Magali.

In [ ]:
import numpy as np
import pandas as pd
from fair import FAIR
from fair.interface import fill, initialise
from scipy.interpolate import interp1d
import xarray as xr
import matplotlib.pyplot as pl

In [ ]:
ghg = pd.read_csv('../data/concentration/ch4_n2o_concentrations_1750-1901.csv', index_col='time')

In [ ]:
concentrations = np.zeros((152, 2))
concentrations[0, :] = ghg.loc[1750.5, :]
interpolator = interp1d(ghg.index, ghg.values, axis=0)
concentrations[1:, :] = interpolator(np.arange(1751, 1902))
concentrations = pd.DataFrame(
    concentrations,
    columns = ['CH4', 'N2O'],
    index = np.arange(1750, 1902)
)
concentrations

In [ ]:
ghg.values.shape

In [ ]:
da_emissions = xr.load_dataarray(
    "../data/emissions/ssp_emissions_1750-2500.nc"
)

In [ ]:
df_configs = pd.read_csv(
    "../data/fair2.1-parameters/calibration-1.4.2/calibrated_constrained_parameters.csv",
    index_col=0,
)
valid_all = df_configs.index

In [ ]:
emissions = da_emissions.sel(scenario='ssp119', specie=['Sulfur', 'BC', 'OC', 'NH3', 'CO', 'VOC', 'NOx'], timepoints=np.arange(1750.5, 1901), config='unspecified')

In [ ]:
f = FAIR(temperature_prescribed=True, ghg_method='myhre1998')

In [ ]:
f.define_time(1750, 1901, 1)
f.define_scenarios(['ssp119'])
f.define_configs(valid_all)

In [ ]:
species = ['Sulfur', 'BC', 'OC', 'NH3', 'CO', 'VOC', 'NOx', 'CH4', 'N2O', 'Aerosol-radiation interactions', 'Aerosol-cloud interactions', 'Ozone']

In [ ]:
properties = {
    'CH4': {
        'type': 'ch4',
        'input_mode': 'concentration',
        'greenhouse_gas': True,
        'aerosol_chemistry_from_emissions': False,
        'aerosol_chemistry_from_concentration': True, # we treat methane as a reactive gas
    },
    'N2O': {
        'type': 'n2o',
        'input_mode': 'concentration',
        'greenhouse_gas': True,
        'aerosol_chemistry_from_emissions': False,
        'aerosol_chemistry_from_concentration': True, # we treat nitrous oxide as a reactive gas
    },
    'Sulfur': {
        'type': 'sulfur',
        'input_mode': 'emissions',
        'greenhouse_gas': False,
        'aerosol_chemistry_from_emissions': True,
        'aerosol_chemistry_from_concentration': False,
    },
    'BC': {
        'type': 'black carbon',
        'input_mode': 'emissions',
        'greenhouse_gas': False,
        'aerosol_chemistry_from_emissions': True,
        'aerosol_chemistry_from_concentration': False,
    },
    'OC': {
        'type': 'organic carbon',
        'input_mode': 'emissions',
        'greenhouse_gas': False,
        'aerosol_chemistry_from_emissions': True,
        'aerosol_chemistry_from_concentration': False,
    },
    'VOC': {
        'type': 'other slcf',
        'input_mode': 'emissions',
        'greenhouse_gas': False,
        'aerosol_chemistry_from_emissions': True,
        'aerosol_chemistry_from_concentration': False,
    },
    'NOx': {
        'type': 'other slcf',
        'input_mode': 'emissions',
        'greenhouse_gas': False,
        'aerosol_chemistry_from_emissions': True,
        'aerosol_chemistry_from_concentration': False,
    },
    'NH3': {
        'type': 'other slcf',
        'input_mode': 'emissions',
        'greenhouse_gas': False,
        'aerosol_chemistry_from_emissions': True,
        'aerosol_chemistry_from_concentration': False,
    },
    'CO': {
        'type': 'other slcf',
        'input_mode': 'emissions',
        'greenhouse_gas': False,
        'aerosol_chemistry_from_emissions': True,
        'aerosol_chemistry_from_concentration': False,
    },
    'Aerosol-radiation interactions': {
        'type': 'ari',
        'input_mode': 'calculated',
        'greenhouse_gas': False,
        'aerosol_chemistry_from_emissions': False,
        'aerosol_chemistry_from_concentration': False,
    },
    'Aerosol-cloud interactions': {
        'type': 'aci',
        'input_mode': 'calculated',
        'greenhouse_gas': False,
        'aerosol_chemistry_from_emissions': False,
        'aerosol_chemistry_from_concentration': False,
    },
    'Ozone': {
        'type': 'ozone',
        'input_mode': 'calculated',
        'greenhouse_gas': False,
        'aerosol_chemistry_from_emissions': False,
        'aerosol_chemistry_from_concentration': False,
    }
}

In [ ]:
# fair.structure.species.species_types
# move to doc!

In [ ]:
f.define_species(species, properties)

In [ ]:
f.allocate()

In [ ]:
for specie in ['Sulfur', 'BC', 'OC', 'NH3', 'CO', 'VOC', 'NOx']:
    f.emissions.loc[dict(specie=specie)] = emissions.sel(specie=specie)

In [ ]:
for specie in ['CH4', 'N2O']:
    f.concentration.loc[dict(specie=specie)] = np.array(concentrations.loc[1750:1901, specie])[:, None, None]

In [ ]:
fill(
    f.climate_configs["ocean_heat_capacity"],
    df_configs.loc[:, "clim_c1":"clim_c3"].values,
)
fill(
    f.climate_configs["ocean_heat_transfer"],
    df_configs.loc[:, "clim_kappa1":"clim_kappa3"].values,
)  # not massively robust, since relies on kappa1, kappa2, kappa3 being in adjacent cols
fill(
    f.climate_configs["deep_ocean_efficacy"],
    df_configs.loc[:, "clim_epsilon"],
)
fill(
    f.climate_configs["gamma_autocorrelation"],
    df_configs.loc[:, "clim_gamma"],
)
fill(f.climate_configs["sigma_eta"], df_configs.loc[:, "clim_sigma_eta"])
fill(f.climate_configs["sigma_xi"], df_configs.loc[:, "clim_sigma_xi"])
#fill(f.climate_configs["seed"], df_configs["seed"])
fill(f.climate_configs["stochastic_run"], False)
#fill(f.climate_configs["stochastic_run"], True)
#fill(f.climate_configs["use_seed"], True)
fill(f.climate_configs["forcing_4co2"], df_configs.loc[:, "clim_F_4xCO2"])

In [ ]:
# species level
f.fill_species_configs()

# aerosol indirect
fill(f.species_configs["aci_scale"], df_configs.loc[:, "aci_beta"])
fill(
    f.species_configs["aci_shape"],
    df_configs.loc[:, "aci_shape_so2"],
    specie="Sulfur",
)
fill(
    f.species_configs["aci_shape"],
    df_configs.loc[:, "aci_shape_bc"],
    specie="BC",
)
fill(
    f.species_configs["aci_shape"],
    df_configs.loc[:, "aci_shape_oc"],
    specie="OC",
)

# emissions adjustments for NOx
fill(f.species_configs["baseline_emissions"], 19.423526730206152, specie="NOx")

# aerosol direct
for specie in [
    "BC",
    "CH4",
    "N2O",
    "NH3",
    "NOx",
    "OC",
    "Sulfur",
    "VOC",
]:
    fill(
        f.species_configs["erfari_radiative_efficiency"],
        df_configs.loc[:, f"ari_{specie}"],
        specie=specie,
    )

# ozone
for specie in [
    "CH4",
    "N2O",
    "CO",
    "VOC",
    "NOx",
]:
    fill(
        f.species_configs["ozone_radiative_efficiency"],
        df_configs.loc[:, f"o3_{specie}"],
        specie=specie,
    )

# initial conditions
initialise(f.forcing, 0)
fill(f.temperature, 0)

In [ ]:
f.run()

In [ ]:
pl.plot(
    f.timebounds,
    (
        f.forcing.sel(specie='Aerosol-radiation interactions', scenario='ssp119') +
        f.forcing.sel(specie='Aerosol-cloud interactions', scenario='ssp119')
    ).median(dim='config')
);

In [ ]:
pl.plot(
    f.timebounds,
    f.forcing.sel(specie='Ozone', scenario='ssp119').median(dim='config')
);